Colorado updates their data daily at 4pm MDT for cases through 4pm prior day.

In [26]:
from selenium import webdriver
import time
import pandas as pd
import pendulum
import re
import yaml
import requests
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox) # linux only
# chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("user-agent=[Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:73.0) Gecko/20100101 Firefox/73.0]")

In [ ]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f.read())

In [ ]:
state = 'CO'

In [8]:
scrape_timestamp = pendulum.now().strftime('%Y%m%d%H%M%S')

In [5]:
# Colorado exposes their data as ArcGIS geojson via an API
url = 'https://opendata.arcgis.com/datasets/fbae539746324ca69ff34f086286845b_0.geojson'

In [2]:
def fetch():
#     driver = webdriver.Chrome('../20190611 - Parts recommendation/chromedriver', options=chrome_options)

#     driver.get(url)
#     time.sleep(5)

#     # class topBoxH1Text is used in the summary box.
#     data_elements = driver.find_elements_by_class_name('topBoxH1Text')

#     datatbl = data_elements[1].text.split('\n')

#     data = [re.search('^(.*) - (\d*)', row) for row in datatbl]

#     data = [row.groups() for row in data if row]
#     page_source = driver.page_source
#     driver.close()
    
    # Colorado exposes their data as ArcGIS geojson via an API
    res = requests.get(url).json()
    data = [ele.pop('properties', None) for ele in res['features']]
    df = pd.DataFrame.from_dict(data)
    
    # adding standard column names
    df['county'] = df.COUNTY
    df['positive_cases'] = df.County_Pos_Cases
    
    return df, str(res)

In [1]:
def save(df, source):
    df.to_csv(f"{config['data_folder']}/{state}_county_{scrape_timestamp}.txt", sep='|', index=False)

    with open(f"{config['data_source_backup_folder']}/{state}_county_{scrape_timestamp}.json", 'w') as f:
        f.write(source)

In [ ]:
def run():
    df, source = fetch()
    save(df, source)